# Tensor products and contraction

#### Test: Product of two tensors creates new tensor with higher rank

```python
    def test_tensor_product_returns_new_tensor(self):
        # Arrange
        r, phi = sp.symbols('r, phi')
        metric = sp.diag(1, r ** 2)
        plane = Manifold(metric, coords=(r, phi))
        A = Tensor(plane, 'ulu')
        B = Tensor(plane, 'lu')

        # Act
        C = A * B

        # Assert
        self.assertEqual(C.rank, 5)

```

```
TypeError: unsupported operand type(s) for *: 'Tensor' and 'Tensor'
```


Implement `Tensor.__mul__`:

```python
    def __mul__(self, other):
        result = Tensor(self.manifold, self.idx_pos + other.idx_pos, values=self.values)

        for m_idx in result.values:
            result[m_idx] *= other[m_idx]

        return result
```

Error:

```
AttributeError: 'Tensor' object has no attribute 'rank'
```

Add to `Tensor` read-only attribute:

```python
    @property
    def rank(self):
        return len(self.idx_pos)

```

## Contraction

#### Test: Contraction of tensor on two indices returns new tensor of lower rank

```python
    def test_tensor_contraction_returns_new_tensor_of_lower_rank(self):
        # Arrange
        coords = sp.symbols('t, x, y, z')
        metric = sp.diag(-1, 1, 1, 1)
        minkowski = Manifold(metric, coords)
        x = Tensor(minkowski, 'u', values={k: c for k, c in enumerate(coords)})
        eta = Tensor(minkowski, 'll', {(k, k): metric[k, k] for k in range(4)})

        # Act
        x_low = (eta * x).contract(1, 2)

        # Assert
        self.assertEqual(x_low.rank, 1)
        self.assertEqual(x_low.idx_pos, 'l')


```

Error:

```
File "/Users/mbaer/PycharmProjects/diffgeom/diffgeom/diffgeom.py", line 37, in _translate_multi_idx
    for idx in m_idx:
TypeError: 'int' object is not iterable
```

Error is unexpected => Found a bug!

Modify:

```python

    def _translate_multi_idx(self, m_idx):
        result = []
        if not hasattr(m_idx, '__len__'):
            m_idx = (m_idx, )
        for idx in m_idx:
            if isinstance(idx, int):
                result.append(idx)
            else:
                result.append(self.names_map[idx])
        return tuple(result)
```

Error:

```
AttributeError: 'Tensor' object has no attribute 'contract'
```

This is what we expect. Implement!

```python

    def contract(self, idx1, idx2):
        if idx2 < idx1:
            idx1, idx2 = idx2, idx1
        idx_pos = list(self.idx_pos)
        del idx_pos[idx2]
        del idx_pos[idx1]
        idx_pos = ''.join(idx_pos)

        result = Tensor(self.manifold, idx_pos)

        for m_idx in self.multi_indices:
            if m_idx[idx1] != m_idx[idx2]:
                continue
            value = self[m_idx]
            m_idx = list(m_idx)
            del m_idx[idx2]
            del m_idx[idx1]
            result[m_idx] += value

        return result

    @property
    def multi_indices(self):
        from itertools import product
        return product(*tuple(list(range(len(self.manifold.coords))) for _ in range(self.rank)))
    
```

A bit hacky. Plus: bug found in IndexedObject.\__init__:

```python
        if values is None:
            self.values = {}
        else:
            self.values = dict(values)
```

Note the dict to force a copy!

#### Test: Lowering an index returns tensor with other structure

```python
    def test_lowering_index_returns_lowered_tensor(self):
        # Arrange
        coords = sp.symbols('t, x, y, z')
        metric = sp.diag(-1, 1, 1, 1)
        minkowski = Manifold(metric, coords)
        xx = Tensor(minkowski, 'uu', values={(k,k): c for k, c in enumerate(coords)})

        # Act
        xx_low = xx.lower_index(0)

        # Assert
        self.assertEqual(xx_low.idx_pos, 'lu')
        self.assertEqual(xx_low[0, 0], -coords[0])
        self.assertEqual(xx_low[1, 1], coords[1])
```

```python

    def lower_index(self, idx):
        idx_pos = list(self.idx_pos)
        if idx_pos[idx] == 'l':
            raise IncompatibleIndexPositionException('Index already downstairs.')
        idx_pos[idx] = 'l'
        result = Tensor(self.manifold, str(idx_pos))
        g = self.manifold.metric
        g = Tensor(self.manifold, 'll', values={(k, k): g[k, k] for k in range(self.manifold.dims)})
        return (g * self).contract(1, 2 + idx)
```


#### Test:     

```python
def test_raising_index_returns_raised_tensor(self):

        # Arrange
        coords = sp.symbols('t, x, y, z')
        metric = sp.diag(-1, 1, 1, 1)
        minkowski = Manifold(metric, coords)
        xx = Tensor(minkowski, 'll', values={(k,k): c for k, c in enumerate(coords)})
        xx[0, 0] *= -1

        # Act
        xx_high = xx.raise_index(0)

        # Assert
        self.assertEqual(xx_high.idx_pos, 'ul')
        self.assertEqual(xx_high[0, 0], coords[0])
        self.assertEqual(xx_high[1, 1], coords[1])
```


#### Test: Covariant derivative returns tensor of one rank higher

```python

    def test_diff_tensor(self):
        r, ph, th = sp.symbols('r, phi, theta')
        sphere = Manifold(sp.diag(r ** 2, r ** 2 * sp.sin(th) ** 2), (th, ph))

        A = Tensor(sphere, 'ul', {(0, 0): th**2+ph**3,
                                    (0, 1): th**3 + ph**2,
                                    (1, 0): th**2 * ph**3,
                                    (1, 1): th**3 * ph**2
                                    })

        nabla_A = A.diff()
        C = sphere.gammas

        assert nabla_A[(th, th, th)] == sp.diff(A[th, th], th)
        assert nabla_A[(th, th, ph)] ==  sp.diff(A[th, th], ph) + A[th, th] * C[th, th, ph] + A[ph, th] * C[th, ph, ph] \
                                        - A[th, th] * C[th, th, ph] - A[th, ph] * C[ph, th, ph]
        assert nabla_A[(th, th, ph)] != sp.diff(A[th, th], ph)

```

#### Test: Riemann tensor of Euclidean plane in polar coordinates vanishes

```python


class TestRiemann(unittest.TestCase):

    def test_euclidean_plan_has_vanishing_riemann(self):

        r, phi = s.symbols('r phi')
        metric = s.diag(1, r**2)
        plane = Manifold(metric, (r, phi))

        riemann = RiemannTensor(plane)
        assert len(riemann.values) == 0

```

#### Test: Sphere has nonvanishing Riemann tensor

```python


    def test_sphere_has_nonvanishing_riemann(self):

        r, ph, th = s.symbols('r, phi, theta')
        sphere = Manifold(s.diag(r ** 2, r ** 2 * s.sin(th) ** 2), (th, ph))
        R = RiemannTensor(sphere)

        assert len(R.values) > 0
        assert R[0, 1, 0, 1] == s.sin(th)**2
        assert R[0, 1, 1, 0] == -s.sin(th) ** 2
        assert R[1, 0, 1, 0] == 1
        assert R[1, 0, 0, 1] == -1
        
```

$$
\rm{\Box}
$$